1.决策树思想

信息熵，概率p(x)如果等于1，信息熵 = -p(x)logp(x)=0

根据例子讲述ID3算法

手推C4.5（信息增益率，考虑样本数量，考虑由于样本多带来的信息增益大，加一个惩罚系数，不根据样本量拆分）

惩罚项 = 子分类/总类

2.基于西瓜数据，自己写ID3与C4.5算法

CART算法：分类树（gini基尼系数），回归树（最小二乘树，二分法）

分别自己用ID3，C4.5,CART算法算西瓜数据

3.通过验证集判断是否为过拟合

预剪枝：贪心算法，没有一定的信息增益就不要长出来这个树枝，不进行拆分。树的最大深度不能太多，每次分信息增益率必须达到一定的值，训练集拆出来验证集，用验证集验证时如果存在准确率下降的情况，也要进行预剪枝。

后剪枝：训练集产生出来树，用测试集去测试哪些枝是没必要长出来的。

bagging：随机森林，弱分类器通过投票的方式构建一个强分类器

4.给出伪代码，实现决策树

In [22]:
import numpy as np
import pandas as pd
import math

In [23]:
data = pd.read_csv('./西瓜数据.txt')
data

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,0.243,0.267,否


# ID3

In [24]:
def info(x,y):
    if x != y and x != 0:
        info_X = -(x/y)*math.log2(x/y)-((y-x)/y)*math.log2((y-x)/y)
    if x == y or x == 0:
        info_X = 0
    return info_X

In [25]:
info_D = info(8,17)
info_D

0.9975025463691152

In [26]:
dataset[dataset.色泽=='乌黑']

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
14,15,乌黑,稍蜷,浊响,清晰,稍凹,软粘,0.360,0.370,否


In [27]:
#色泽信息增益：青绿6,3好，乌黑6，4好，浅白5，1好
info_col = 6/17*info(3,6)+ 6/17*info(4,6)+5/17*info(1,5)
gain_col = info_D - info_col
gain_col

0.1081251652653652

In [28]:
data[data.根蒂 =='硬挺']

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,0.243,0.267,否
10,11,浅白,硬挺,清脆,模糊,平坦,硬滑,0.245,0.057,否


In [29]:
#根蒂信息增益：
info_root = 8/17*info(5,8)+7/17*info(3,7)+2/17*info(0,2)
gain_root = info_D - info_root
gain_root

0.14267495956679277

In [30]:
data[data.敲声 =='沉闷']

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
13,14,浅白,稍蜷,沉闷,稍糊,凹陷,硬滑,0.657,0.198,否
16,17,青绿,蜷缩,沉闷,稍糊,稍凹,硬滑,0.719,0.103,否


In [31]:
#敲声信息增益：
info_sound = 2/17*info(0,2)+10/17*info(6,10)+5/17*info(2,5)
gain_sound = info_D - info_sound
gain_sound

0.14078143361499573

In [32]:
data[data.纹理 =='模糊']

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
10,11,浅白,硬挺,清脆,模糊,平坦,硬滑,0.245,0.057,否
11,12,浅白,蜷缩,浊响,模糊,平坦,软粘,0.343,0.099,否
15,16,浅白,蜷缩,浊响,模糊,平坦,硬滑,0.593,0.042,否


In [33]:
#纹理信息增益：
info_line = 9/17*info(7,9)+5/17*info(1,5)+3/17*info(0,3)
gain_line = info_D - info_line
gain_line

0.3805918973682685

In [34]:
data[data.脐部 =='稍凹']

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
14,15,乌黑,稍蜷,浊响,清晰,稍凹,软粘,0.360,0.370,否
16,17,青绿,蜷缩,沉闷,稍糊,稍凹,硬滑,0.719,0.103,否


In [35]:
#脐部信息增益：
info_but = 4/17*info(0,4)+7/17*info(5,7)+6/17*info(3,6)
gain_but = info_D - info_but
gain_but

0.28915878284167884

In [36]:
data[data.触感 == '硬滑']

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
10,11,浅白,硬挺,清脆,模糊,平坦,硬滑,0.245,0.057,否
12,13,青绿,稍蜷,浊响,稍糊,凹陷,硬滑,0.639,0.161,否
13,14,浅白,稍蜷,沉闷,稍糊,凹陷,硬滑,0.657,0.198,否


In [37]:
#触感信息增益：
info_tou = 5/17*info(2,5)+12/17*info(6,12)
gain_tou = info_D - info_tou
# gain_tou_rate = gain_tou/info_tou
print('触感信息熵，信息增益：',info_tou,gain_tou)

触感信息熵，信息增益： 0.9914560571925497 0.006046489176565473


# 决策树的实现
https://blog.csdn.net/u013719339/article/details/84395406